<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт" data-toc-modified-id="Импорт-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт</a></span></li><li><span><a href="#Описание" data-toc-modified-id="Описание-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Описание</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Классификация-с-помощью-Bag-of-words" data-toc-modified-id="Классификация-с-помощью-Bag-of-words-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Классификация с помощью Bag of words</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регресии" data-toc-modified-id="Модель-логистической-регресии-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Модель логистической регресии</a></span></li><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Модель-градиентного-спуска" data-toc-modified-id="Модель-градиентного-спуска-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Модель градиентного спуска</a></span></li><li><span><a href="#Сравнение-и-проверка-на-тестовой-выборке" data-toc-modified-id="Сравнение-и-проверка-на-тестовой-выборке-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Сравнение и проверка на тестовой выборке</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт

Импортируем необходимые для работы библиотеки. Считаем данные из csv-файла в датафрейм, сохраним в переменную `data` и выведем на экран первые десять строк и общую информацию о датасете.

In [2]:
# miscellaneous
import pickle

# data analysis
import pandas as pd
import numpy as np

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    GridSearchCV, 
    RandomizedSearchCV,
    cross_val_score,
    train_test_split
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
import lightgbm as lgb

# nlp
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import re
import torch
import transformers

In [3]:
# испльзуем опцию index_col, чтобы не создавать дубликат столбца с индексами
try:
    data = pd.read_csv(
        'https://code.s3.yandex.net/datasets/toxic_comments.csv',
        index_col=0)
except:
    data = pd.read_csv(
        'toxic_comments.csv',
        index_col=0)

pd.concat([data.head(5), data.tail(5)])

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0
159450,"""\nAnd ... I really don't think you understand...",0


### Описание

Выведем общую информацию о датасете.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


В датасете 2 признака и 159292 объектов. Типы данных в порядке.

Признаки:
- `text` — текст комментария на английском языке;

Целевой признак:
- `toxic` — является ли комментарий токсичным (1 — да, 0 — нет), бинарный категориальный признак.

In [5]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
toxic,159292.0,0.101612,0.302139,0.0,0.0,0.0,0.0,1.0


In [6]:
data.describe(include=['O']).T

,count,unique,top,freq
text,159292,159292,here are a bunch of other images of the tree o...,1


Пропусков и явных дубликатов нет. значения `toxic` выглядят адекватными. В выборке присутствует дисбаланс классов.

Создадим тренировочную, валидационную и тестовую выборки. Дисбаланс классов учтем при обучении модели.

In [7]:
# сплит на 0.6 и 0.4, 0.6 уходит в переменные train, 0.4 в temp
train, temp = train_test_split(
    data, test_size=0.4, random_state=42, stratify=data['toxic'])

# сплит на 0.5 и 0.5 из переменной temp
test, valid = train_test_split(
    temp, test_size=0.5, random_state=42, stratify=temp['toxic'])

toxic = []
full = []
for i in [train, valid, test]:
    toxic.append(i['toxic'].sum())
    full.append(i.shape[0])

pd.DataFrame(
    {'Количество токсичных комментариев':toxic,
     'Общее количество комментариев':full
    },
    index=['train', 'valid', 'test']
)

,Количество токсичных комментариев,Общее количество комментариев
train,9712,95575
valid,3237,31859
test,3237,31858


Таким образом, перед нами задача классификации. Для ее решения будем подход bag of words для векторизации текста и модели машинного обучения. Метрикой качества будет выступать F1-мера, результат должен быть не менее 0.75.

## Обучение

### Классификация с помощью Bag of words

В начале создадим набор функций для очистки и лемматизации текста.

In [8]:
# загрузка spacy с английским языком
lemm_model = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

'''
Функция принимает текст и лемматизирует его с помощью spacy
'''
def lemmatize(text):
    doc = lemm_model(text)

    return " ".join([token.lemma_ for token in doc])

'''
Функция очищает текст, оставляет только английские буквы с нижним регистром и
пробелы
'''
def clear_text(text):
    text = re.sub(r'[^a-z ]', ' ', text)
    text = text.split()

    return ' '.join(text)

'''
Функция принимает датафрейм с признаком text и возвращает очищенный и
лематизированный корпус
'''
def create_corp(df):
    lowered = df['text'].str.lower() # все буквы к нижнему регистру
    cleared = lowered.apply(clear_text) # очищение
    lemmatized = cleared.apply(lemmatize) # лемматизация

    return lemmatized.values

In [9]:
# использование дампов
try:
    with open('train.bin', 'rb') as f:
        corpus_train = pickle.load(f)
        f.close()
        
    with open('test.bin', 'rb') as f:
        corpus_test = pickle.load(f)
        f.close()
        
    with open('valid.bin', 'rb') as f:
        corpus_valid = pickle.load(f)
        f.close()
        
except:
    corpus_train = create_corp(train)
    corpus_valid = create_corp(valid)
    corpus_test = create_corp(test)

Вычислим TF-IDF для корпуса текстов.

In [10]:
# загрузка стоп-слов
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kirill.V.Gusev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создадим пайплайн для дальнейшей кросс-валидации без утечек данных.

In [11]:
'''
Функция для подбора гиперпараметров.
Принимает модель, вид поиска, признаки,, гиперпараметры для перебора и название модели для отображения.
Строит пайплайн, запускает поиск гиперпараметров, выводит значения гиперпараметров и скор на обучении.
Возвращает лучший пайплайн и лучшую метрику.

search_type:
C - cross_val_score
G - GridSearchCV
R - RandomizedSearchCV
'''
def pipe(model, search_type, X_train, y_train, params, name, cv=3):
    
    # ветка для cross_val_score
    if search_type == "C":      
        pipeline = make_pipeline(count_tf_idf, model)         
        model_score = cross_val_score(
            pipeline,
            X_train,
            y_train,
            scoring='f1',
            cv=cv).mean()
        # значение f1 на кросс-валидации
        print(f'best_score: {model_score.round(3)}')
        storage[name] = [model_score.round(3)]
    
        return pipeline, model_score
    
    # ветка для GridSearchCV
    elif search_type == "G":
        pipeline = make_pipeline(count_tf_idf, model)
        gs = GridSearchCV(
            pipeline, 
            param_grid=params, 
            scoring='f1', 
            n_jobs=-1,
            error_score='raise',
            verbose=2,
            cv=cv
        )        
        gs.fit(X_train, y_train)
        #лучшее значение f1 на кросс-валидации
        print(f'best_score: {gs.best_score_.round(3)}')
        # лучшие гиперпараметры
        print(f'best_params: {gs.best_params_}')  
        storage[name] = [gs.best_score_.round(3)] 
        
        return gs.best_estimator_, gs.best_score_.round(3)
    
    # ветка для RandomizedSearchCV
    elif search_type == "R":
        pipeline = make_pipeline(count_tf_idf, model)
        rs = RandomizedSearchCV(
            pipeline,
            param_distributions=params, 
            scoring='f1', 
            n_jobs=-1,
            random_state=42,
            verbose=2,
            cv=cv
        )
        rs.fit(X_train, y_train)
        # лучшее значение f1 на кросс-валидации
        print(f'best_score: {rs.best_score_.round(3)}')
        # лучшие гиперпараметры
        print(f'best_params: {rs.best_params_}')
        storage[name] = [rs.best_score_.round(3)] 

        return rs.best_estimator_, rs.best_score_.round(3)
        
    else: print('Введите верный search_type')   

#### Модель логистической регресии

In [12]:
%%time
storage = {}
model = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)

param_grid_LR = {
    "tfidfvectorizer__ngram_range": ((1, 1), (1, 2)),
    'logisticregression__C': np.logspace(-1, 2, 7)}

pipeline_LR, score_LR = pipe(
    model,
    'G',
    corpus_train,
    train['toxic'],
    param_grid_LR,
    'Linear Regression'
)

Fitting 3 folds for each of 14 candidates, totalling 42 fits
best_score: 0.787
best_params: {'logisticregression__C': 100.0, 'tfidfvectorizer__ngram_range': (1, 2)}
CPU times: total: 2min 52s
Wall time: 8min 36s


Попробуем улучшить метрику, перебирая пороговые значения.

In [13]:
'''
функция выводит таблицу с пороговыми значениями и метрикой f1
на вход подается обученная модель, признаки, целевой признак, правая граница порогов и шаг
'''
def f1_check(model, features, target, border, step):
    proba = model.predict_proba(features)
    proba_one = proba[:, 1]
    temp_df = pd.DataFrame(columns=['f1_score', 'threshold'])
    for threshold in np.arange(0, border, step):
        predicted = proba_one > threshold
        temp_df = pd.concat(
            [temp_df, pd.DataFrame(
                {'f1_score':f1_score(target, predicted),
                 'threshold':threshold},index=[0])],
            ignore_index=True)

    return temp_df.sort_values(by='f1_score', ascending=False).head()

In [14]:
f1_check(pipeline_LR, corpus_valid, valid['toxic'], 1, 0.01)

,f1_score,threshold
47,0.787074,0.47
50,0.787007,0.50
45,0.786828,0.45
46,0.786718,0.46
43,0.786709,0.43


Результат лучше не стал. Проверим другие модели машинного обучения.

#### Модель случайного леса

In [15]:
%%time
model = RandomForestClassifier(random_state=42, class_weight='balanced')

param_grid_RF = {
    'tfidfvectorizer__ngram_range': ((1, 1), (1, 2)),
    'randomforestclassifier__n_estimators': range(10, 30, 10),
    'randomforestclassifier__max_depth': range(2, 10, 2)
}

pipeline_RF, score_RF = pipe(
    model,
    'G',
    corpus_train,
    train['toxic'],
    param_grid_RF,
    'Random Forest'
)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
best_score: 0.314
best_params: {'randomforestclassifier__max_depth': 8, 'randomforestclassifier__n_estimators': 20, 'tfidfvectorizer__ngram_range': (1, 1)}
CPU times: total: 15.9 s
Wall time: 1min 56s


#### Модель градиентного спуска

In [16]:
%%time

model = lgb.LGBMClassifier(random_state=42, class_weight='balanced')

param_grid_lgbm = {
    'tfidfvectorizer__ngram_range': ((1, 1), (1, 2)),
    'lgbmclassifier__num_leaves': range(10, 50, 10),
    'lgbmclassifier__learning_rate': np.logspace(-1, 0, 5),
}

pipeline_lgbm, score_lgbm = pipe(
    model,
    'G',
    corpus_train,
    train['toxic'],
    param_grid_lgbm,
    "LightGBM")

Fitting 3 folds for each of 40 candidates, totalling 120 fits


C:\Users\Kirill.V.Gusev\.conda\envs\ds_practicum_env\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


best_score: 0.755
best_params: {'lgbmclassifier__learning_rate': 0.31622776601683794, 'lgbmclassifier__num_leaves': 40, 'tfidfvectorizer__ngram_range': (1, 2)}
CPU times: total: 6min 38s
Wall time: 16min 3s


Из каждого датафрейма получим признаки и целевой признак для обучения. Признак 'key_0' можно удалить.

#### Сравнение и проверка на тестовой выборке

Сравним результаты трех моделей.

In [17]:
temp = pd.DataFrame(storage).T
temp.columns = ['Cross-validation Score']
temp.style.format("{:.3f}").highlight_max(color='lightgreen')

,Cross-validation Score
Linear Regression,0.787
Random Forest,0.314
LightGBM,0.755


Лучше всех себя показала модель логистической регресии. Выведем для нее результат на тесте.

In [18]:
%%time

# обучение на увеличенной выборке train + test
giga_train = np.concatenate((corpus_train, corpus_valid))
giga_target = pd.concat([train['toxic'], valid['toxic']])

pipeline_LR.fit(giga_train, giga_target)
prediction = pipeline_LR.predict(corpus_test)
f1_score(test['toxic'], prediction).round(3)

CPU times: total: 3min 45s
Wall time: 2min 33s


0.793

Условие задачи выполняется, метрика выше 0.75.

## Выводы

**Вывод:** В рамках решения задачи были выполнены следующие этапы:
- проведены исследование и предобработка данных,
- текст очищен и лемматизирован,
- использован подход bag of words для конвертации слов в вектора,
- проведена классификация с помощью моделей машинного обучения,
- проведено сравнение резульатов.

По итогам исследования выбрана модель логистической регрессии, так как она показала наилучшую метрику качества (более 75).